# Test notebook

__A. J. Tropiano [tropiano.4@osu.edu]__<br/>
__March 17, 2022__

The purpose of this notebook is for testing out code and/or scripts.

_Last update: April 24, 2022_

In [1]:
# Python imports
from matplotlib.offsetbox import AnchoredText
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import time

In [2]:
# Imports from A.T. codes
import densities
from dmd import Deuteron
from momentum_distributions import MomentumDistribution
from potentials import Potential
# You could import modules but I want a list of scripts here
# Check these off the list as you go through Jupyter notebooks generating
# figures
import modules.a2
import modules.figures
import modules.fourier_transform
from modules.integration import gaussian_quadrature_mesh
import modules.labels
import modules.long_distance_operators
from magnus import Magnus, MagnusSplit
import modules.momentum_projection_operator
import modules.phase_shifts
import modules.sdt
from pmd import Pair
from snmd import SingleNucleon
from srg import SRG, get_transformation
import modules.tools
import modules.wave_function
# add missing ones

For now let's illustrate the overhaul by testing scripts and showing how to use them.

### Testing momentum distributions code

In [3]:
# # Test default interaction and SRG evolution

# kvnn = 6
# kmax, kmid, ntot = 15.0, 3.0, 120
# channels = ('1S0', '3S1')
# generator = 'Wegner'
# lamb = 1.35

In [4]:
# # Get momentum values (channel argument doesn't matter here)
# potential = Potential(kvnn, '1S0', kmax, kmid, ntot)
# q_array, _ = potential.load_mesh()

# # Set C.o.M. momentum too
# Q_max = 2.0
# ntot_Q = 40
# Q_array, _ = gaussian_quadrature_mesh(Q_max, ntot_Q)

In [5]:
# # Do Ca48 with SLy4 densities

# nucleus_name = 'Ca48'
# Z, N = 20, 28
# density = 'SLY4'

In [6]:
# # Generate new files and compare to old ones
# # Single-nucleon matches old data exactly

# snmd = SingleNucleon(kvnn, kmax, kmid, ntot, channels, generator, lamb)

# for nucleon in ('proton', 'neutron'):
    
#     t0 = time.time()
    
#     n_array = snmd.compute_momentum_distribution(
#         q_array, nucleon, nucleus_name, Z, N, density, save=True)
    
#     t1 = time.time()
#     mins = (t1-t0)/60
#     print(f'Done with {nucleon} after {mins:.2f} minutes.')

In [7]:
# # Pair matches old data exactly

# pmd = Pair(kvnn, kmax, kmid, ntot, channels, generator, lamb)

# for pair in ('pn', 'pp', 'nn'):
    
#     t0 = time.time()
            
#     # Q > 0
#     n_array = pmd.compute_momentum_distribution(
#         q_array, Q_array, pair, nucleus_name, Z, N, density, save=True)
    
#     # Q = 0
#     n_Q0_array = pmd.compute_momentum_distribution_Q0(
#         q_array, pair, nucleus_name, Z, N, density, save=True)

#     t1 = time.time()
#     mins = (t1-t0)/60
#     print(f'Done with {pair} after {mins:.2f} minutes.')

In [8]:
# # Deuteron matches old data exactly

# dmd = Deuteron(kvnn, kmax, kmid, ntot, generator, lamb)
    
# t0 = time.time()
        
# n_array = dmd.compute_momentum_distribution(q_array, save=True)
            
# t1 = time.time()
# mins = (t1-t0)/60
        
# print(f'Done after {mins:.2f} minutes.')

### Generate momentum distribution data

In [9]:
# kvnns = (2, 3, 4, 5, 6, 7, 79, 110, 111, 112, 113, 222, 224)
# # Note: Each kvnn + Gogny run takes about 2.7 hours
# kmax, kmid, ntot = 15.0, 3.0, 120
# channels = ('1S0', '3S1')
# generator = 'Wegner'
# lamb = 1.35
# density = 'Gogny'
# nuclei_gogny = (
#     ('He4', 2, 2), ('Li7', 3, 4), ('Be9', 4, 5), ('C12', 6, 6), ('O16', 8, 8),
#     ('Al27', 13, 14), ('Ca40', 20, 20), ('Ca48', 20, 28), ('Ti48', 22, 26),
#     ('Fe56', 26, 30), ('Cu63', 29, 34), ('Ag107', 47, 60), ('Sn118', 50, 68),
#     ('Ce140', 58, 82), ('Ta181', 73, 108), ('Au197', 79, 118),
#     ('Pb208', 82, 126), ('U238', 92, 146)
# )

# potential = Potential(6, '1S0', kmax, kmid, ntot)
# q_array, _ = potential.load_mesh()  # This will be the same in every case

# # Set C.o.M. momentum too
# Q_max = 2.0
# ntot_Q = 40
# Q_array, _ = gaussian_quadrature_mesh(Q_max, ntot_Q)

# # Generate files
# for kvnn in kvnns:
    
#     print(f'Starting kvnn = {kvnn}.')
#     t0_kvnn = time.time()

#     dmd = Deuteron(kvnn, kmax, kmid, ntot, generator, lamb)
#     snmd = SingleNucleon(kvnn, kmax, kmid, ntot, channels, generator, lamb)
#     pmd = Pair(kvnn, kmax, kmid, ntot, channels, generator, lamb)

#     # Deuteron first
#     t0_d = time.time()
#     n_array = dmd.compute_momentum_distribution(q_array, save=True)   
#     t1_d = time.time()
#     mins = (t1_d-t0_d)/60
#     print(f'Done with deuteron after {mins:.2f} minutes.')
    
#     for nucleus in nuclei_gogny:
        
#         nucleus_name = nucleus[0]
#         Z = nucleus[1]
#         N = nucleus[2]
        
#         t0_nucleus = time.time()
        
#         # Single-nucleon
#         for nucleon in ('proton', 'neutron'):
            
#             n_array = snmd.compute_momentum_distribution(
#                 q_array, nucleon, nucleus_name, Z, N, density, save=True)
        
#         # Pair
#         for pair in ('pn', 'pp', 'nn'):
        
#             # Q > 0
#             n_array = pmd.compute_momentum_distribution(
#                 q_array, Q_array, pair, nucleus_name, Z, N, density,
#                 save=True)
#             # Q = 0
#             n_Q0_array = pmd.compute_momentum_distribution_Q0(
#                     q_array, pair, nucleus_name, Z, N, density, save=True)

#         t1_nucleus = time.time()
#         mins = (t1_nucleus-t0_nucleus)/60
#         print(f'Done with {nucleus_name} after {mins:.2f} minutes.')
        
#     t1_kvnn = time.time()
#     hours = (t1_kvnn-t0_kvnn)/3600
#     print(f'Done with kvnn = {kvnn} after {hours:.2f} hours.\n')

In [10]:
# # Do additional two densities (SLy4 and VMC) for AV18

# kvnn = 6
# kmax, kmid, ntot = 15.0, 3.0, 120
# channels = ('1S0', '3S1')
# generator = 'Wegner'
# lamb = 1.35
# density_list = ['VMC', 'SLy4']
# nuclei_vmc = (('He4', 2, 2), ('He8', 2, 6), ('Be9', 4, 5))
# nuclei_sly4 = (
#     ('He4', 2, 2), ('C12', 6, 6), ('O16', 8, 8), ('Ca40', 20, 20),
#     ('Ca48', 20, 28), ('Fe56', 26, 30), ('Pb208', 82, 126)
# )

# potential = Potential(6, '1S0', kmax, kmid, ntot)
# q_array, _ = potential.load_mesh()  # This will be the same in every case

# # Set C.o.M. momentum too
# Q_max = 2.0
# ntot_Q = 40
# Q_array, _ = gaussian_quadrature_mesh(Q_max, ntot_Q)

# # Generate files
# for density in density_list:
    
#     print(f'Starting density = {density}.')
#     t0_density = time.time()

#     snmd = SingleNucleon(kvnn, kmax, kmid, ntot, channels, generator, lamb)
#     pmd = Pair(kvnn, kmax, kmid, ntot, channels, generator, lamb)
    
#     if density == 'SLy4':
#         nuclei = nuclei_sly4
#     elif density == 'VMC':
#         nuclei = nuclei_vmc
        
#     for nucleus in nuclei:
        
#         nucleus_name = nucleus[0]
#         Z = nucleus[1]
#         N = nucleus[2]
        
#         t0_nucleus = time.time()
        
#         # Single-nucleon
#         for nucleon in ('proton', 'neutron'):
            
#             n_array = snmd.compute_momentum_distribution(
#                 q_array, nucleon, nucleus_name, Z, N, density, save=True)
        
#         # Pair
#         for pair in ('pn', 'pp', 'nn'):
        
#             # Q > 0
#             n_array = pmd.compute_momentum_distribution(
#                 q_array, Q_array, pair, nucleus_name, Z, N, density,
#                 save=True)
#             # Q = 0
#             n_Q0_array = pmd.compute_momentum_distribution_Q0(
#                     q_array, pair, nucleus_name, Z, N, density, save=True)

#         t1_nucleus = time.time()
#         mins = (t1_nucleus-t0_nucleus)/60
#         print(f'Done with {nucleus_name} after {mins:.2f} minutes.')
        
#     t1_density = time.time()
#     hours = (t1_density-t0_density)/3600
#     print(f'Done with density = {density} after {hours:.2f} hours.\n')

In [11]:
# # Do other iterations of channels with AV18 and SLy4
# # Single channel contributions just do SLy4 nuclear proton distributions

# kvnn = 6
# kmax, kmid, ntot = 15.0, 3.0, 120
# channels_total = ('1S0', '3S1', '3P0', '1P1', '3P1')
# generator = 'Wegner'
# lamb = 1.35
# density = 'SLy4'
# nuclei_sly4 = (
#     ('He4', 2, 2), ('C12', 6, 6), ('O16', 8, 8), ('Ca40', 20, 20),
#     ('Ca48', 20, 28), ('Fe56', 26, 30), ('Pb208', 82, 126)
# )

# potential = Potential(6, '1S0', kmax, kmid, ntot)
# q_array, _ = potential.load_mesh()  # This will be the same in every case

# # Set C.o.M. momentum too
# Q_max = 2.0
# ntot_Q = 40
# Q_array, _ = gaussian_quadrature_mesh(Q_max, ntot_Q)

# # First do all partial wave channels combined
# print('Starting all partial wave channels.')
# t0_all = time.time()

# snmd = SingleNucleon(kvnn, kmax, kmid, ntot, channels_total, generator, lamb)
# pmd = Pair(kvnn, kmax, kmid, ntot, channels_total, generator, lamb)

# for nucleus in nuclei_sly4:
    
#     nucleus_name = nucleus[0]
#     Z = nucleus[1]
#     N = nucleus[2]
    
#     t0_nucleus = time.time()
        
#     # Single-nucleon
#     for nucleon in ('proton', 'neutron'):
            
#         n_array = snmd.compute_momentum_distribution(
#             q_array, nucleon, nucleus_name, Z, N, density, save=True)
        
#     # Pair
#     for pair in ('pn', 'pp', 'nn'):
        
#         # Q > 0
#         n_array = pmd.compute_momentum_distribution(
#             q_array, Q_array, pair, nucleus_name, Z, N, density, save=True)
#         # Q = 0
#         n_Q0_array = pmd.compute_momentum_distribution_Q0(
#             q_array, pair, nucleus_name, Z, N, density, save=True)

#     t1_nucleus = time.time()
#     mins = (t1_nucleus-t0_nucleus)/60
#     print(f'Done with {nucleus_name} after {mins:.2f} minutes.')
    
# t1_all = time.time()
# mins = (t1_all-t0_all)/60
# print(f'Done with all partial wave channels after {mins:.2f} minutes.\n')

# # Loop over single partial wave contributions doing only proton distributions
# for channel in channels_total:
    
#     channels = [channel]
    
#     print(f'Starting {channel} channel.')
#     t0_channel = time.time()

#     snmd = SingleNucleon(kvnn, kmax, kmid, ntot, channels, generator, lamb)

#     for nucleus in nuclei_sly4:
        
#         nucleus_name = nucleus[0]
#         Z = nucleus[1]
#         N = nucleus[2]
    
#         t0_nucleus = time.time()
 
#         n_array = snmd.compute_momentum_distribution(
#             q_array, 'proton', nucleus_name, Z, N, density, save=True)

#         t1_nucleus = time.time()
#         mins = (t1_nucleus-t0_nucleus)/60
#         print(f'Done with {nucleus_name} after {mins:.2f} minutes.')
    
#     t1_channel = time.time()
#     mins = (t1_channel-t0_channel)/60
#     print(f'Done with {channel} channel after {mins:.2f} minutes.\n')

In [12]:
# Do inverse-SRG with \delta \lambda = (6, 5.5, ..., 3.5, 3.0) for kvnn and
# kvnn_inv pairs of (111, 6), (111, 113), (113, 6), (222, 6)

kvnn_pairs = ((111, 6), (111, 113), (113, 6), (222, 6))
kmax, kmid, ntot = 15.0, 3.0, 120
channels = ('1S0', '3S1')
generator = 'Wegner'
lamb = 1.35
density = 'Gogny'
nuclei_gogny = (
    ('He4', 2, 2), ('Li7', 3, 4), ('Be9', 4, 5), ('C12', 6, 6), ('O16', 8, 8),
    ('Al27', 13, 14), ('Ca40', 20, 20), ('Ca48', 20, 28), ('Ti48', 22, 26),
    ('Fe56', 26, 30), ('Cu63', 29, 34), ('Ag107', 47, 60), ('Sn118', 50, 68),
    ('Ce140', 58, 82), ('Ta181', 73, 108), ('Au197', 79, 118),
    ('Pb208', 82, 126), ('U238', 92, 146)
)
delta_lambdas = np.arange(6, 2.5, -0.5)

potential = Potential(6, '1S0', kmax, kmid, ntot)
q_array, _ = potential.load_mesh()  # This will be the same in every case

# Set C.o.M. momentum too
Q_max = 2.0
ntot_Q = 40
Q_array, _ = gaussian_quadrature_mesh(Q_max, ntot_Q)

# Generate files
for kvnn_pair in kvnn_pairs:
    
    kvnn_soft, kvnn_hard = kvnn_pair[0], kvnn_pair[1]
    
    print(f'Starting kvnn_soft = {kvnn_soft} and kvnn_hard = {kvnn_hard}.')
    t0 = time.time()

    for dlamb in delta_lambdas:
        
        t0_dlamb = time.time()
        
        dmd = Deuteron(kvnn_soft, kmax, kmid, ntot, generator, lamb,
                       kvnn_inv=kvnn_hard, delta_lambda=dlamb)
        pmd = Pair(kvnn_soft, kmax, kmid, ntot, channels, generator, lamb,
                   kvnn_inv=kvnn_hard, delta_lambda=dlamb)

        # Deuteron first
        t0_d = time.time()
        n_array = dmd.compute_momentum_distribution(q_array, save=True)   
        t1_d = time.time()
        mins = (t1_d-t0_d)/60
        print(f'Done with deuteron after {mins:.2f} minutes.')
    
        for nucleus in nuclei_gogny:
        
            nucleus_name = nucleus[0]
            Z = nucleus[1]
            N = nucleus[2]
        
            t0_nucleus = time.time()
            # pn pair with Q > 0
            n_array = pmd.compute_momentum_distribution(
                q_array, Q_array, 'pn', nucleus_name, Z, N, density,
                save=True)
            t1_nucleus = time.time()
            mins = (t1_nucleus-t0_nucleus)/60
            print(f'Done with {nucleus_name} after {mins:.2f} minutes.')
            
        t1_dlamb = time.time()
        mins = (t1_dlamb-t0_dlamb)/60
        print(f'Done with \delta \lambda = {dlamb} after {mins:.2f}'
              ' minutes.\n')
        
    t1 = time.time()
    hours = (t1-t0)/3600
    print(f'Done with kvnn_soft = {kvnn_soft} and kvnn_hard = {kvnn_hard}'
          f' after {hours:.2f} hours.\n')

Starting kvnn_soft = 111 and kvnn_hard = 6.
Done with deuteron after 0.84 minutes.
Done with He4 after 1.85 minutes.
Done with Li7 after 1.73 minutes.
Done with Be9 after 1.85 minutes.
Done with C12 after 1.92 minutes.
Done with O16 after 1.89 minutes.
Done with Al27 after 1.87 minutes.
Done with Ca40 after 1.87 minutes.
Done with Ca48 after 1.88 minutes.
Done with Ti48 after 1.89 minutes.
Done with Fe56 after 1.87 minutes.
Done with Cu63 after 1.92 minutes.
Done with Ag107 after 1.91 minutes.
Done with Sn118 after 1.96 minutes.
Done with Ce140 after 1.92 minutes.
Done with Ta181 after 1.95 minutes.
Done with Au197 after 1.98 minutes.
Done with Pb208 after 1.94 minutes.
Done with U238 after 1.97 minutes.
Done with \delta \lambda = 6.0 after 35.29 minutes.

Done with deuteron after 0.86 minutes.
Done with He4 after 2.00 minutes.
Done with Li7 after 1.93 minutes.
Done with Be9 after 1.91 minutes.
Done with C12 after 1.89 minutes.
Done with O16 after 1.88 minutes.
Done with Al27 after 1.8

Done with Be9 after 1.93 minutes.
Done with C12 after 1.94 minutes.
Done with O16 after 1.92 minutes.
Done with Al27 after 1.92 minutes.
Done with Ca40 after 1.87 minutes.
Done with Ca48 after 1.86 minutes.
Done with Ti48 after 1.85 minutes.
Done with Fe56 after 1.88 minutes.
Done with Cu63 after 1.90 minutes.
Done with Ag107 after 1.86 minutes.
Done with Sn118 after 1.85 minutes.
Done with Ce140 after 1.86 minutes.
Done with Ta181 after 1.99 minutes.
Done with Au197 after 1.98 minutes.
Done with Pb208 after 1.96 minutes.
Done with U238 after 1.97 minutes.
Done with \delta \lambda = 4.0 after 35.31 minutes.

Done with deuteron after 0.86 minutes.
Done with He4 after 2.01 minutes.
Done with Li7 after 1.93 minutes.
Done with Be9 after 1.92 minutes.
Done with C12 after 1.93 minutes.
Done with O16 after 1.90 minutes.
Done with Al27 after 1.88 minutes.
Done with Ca40 after 1.88 minutes.
Done with Ca48 after 1.86 minutes.
Done with Ti48 after 1.86 minutes.
Done with Fe56 after 1.88 minutes.


Done with C12 after 1.91 minutes.
Done with O16 after 1.91 minutes.
Done with Al27 after 1.95 minutes.
Done with Ca40 after 1.93 minutes.
Done with Ca48 after 1.94 minutes.
Done with Ti48 after 1.95 minutes.
Done with Fe56 after 1.98 minutes.
Done with Cu63 after 1.87 minutes.
Done with Ag107 after 1.87 minutes.
Done with Sn118 after 1.87 minutes.
Done with Ce140 after 2.05 minutes.
Done with Ta181 after 2.03 minutes.
Done with Au197 after 2.13 minutes.
Done with Pb208 after 2.22 minutes.
Done with U238 after 2.05 minutes.
Done with \delta \lambda = 5.5 after 36.43 minutes.

Done with deuteron after 0.87 minutes.
Done with He4 after 2.29 minutes.
Done with Li7 after 2.24 minutes.
Done with Be9 after 2.18 minutes.
Done with C12 after 2.13 minutes.
Done with O16 after 1.98 minutes.
Done with Al27 after 1.89 minutes.
Done with Ca40 after 1.87 minutes.
Done with Ca48 after 1.90 minutes.
Done with Ti48 after 1.81 minutes.
Done with Fe56 after 1.82 minutes.
Done with Cu63 after 1.84 minutes.

In [13]:
# # Do block-diagonal and \lambda dependence for AV18 with Gogny density
# # Don't repeat \lambda = 1.35 for band-diagonal

# kvnn = 6
# kmax, kmid, ntot = 15.0, 3.0, 120
# channels = ('1S0', '3S1')
# generators = ('Wegner', 'Block-diag')
# lambdas = (6.0, 3.0, 1.5, 1.35)
# density = 'SLy4'
# nuclei_sly4 = (
#     ('He4', 2, 2), ('C12', 6, 6), ('O16', 8, 8), ('Ca40', 20, 20),
#     ('Ca48', 20, 28), ('Fe56', 26, 30), ('Pb208', 82, 126)
# )

# potential = Potential(6, '1S0', kmax, kmid, ntot)
# q_array, _ = potential.load_mesh()  # This will be the same in every case

# # Set C.o.M. momentum too
# Q_max = 2.0
# ntot_Q = 40
# Q_array, _ = gaussian_quadrature_mesh(Q_max, ntot_Q)

# # Generate files
# for generator in generators:
    
#     print(f'Starting generator = {generator}.')
#     t0_generator = time.time()
    
#     for lamb in lambdas:
        
#         t0_lamb = time.time()

#         if generator == 'Wegner' and lamb == 1.35:
            
#             break
            
#         else:
            
#             dmd = Deuteron(kvnn, kmax, kmid, ntot, generator, lamb)
#             snmd = SingleNucleon(kvnn, kmax, kmid, ntot, channels, generator,
#                                  lamb)
#             pmd = Pair(kvnn, kmax, kmid, ntot, channels, generator, lamb)

#         # Deuteron first
#         t0_d = time.time()
#         n_array = dmd.compute_momentum_distribution(q_array, save=True)   
#         t1_d = time.time()
#         mins = (t1_d-t0_d)/60
#         print(f'Done with deuteron after {mins:.2f} minutes.')
    
#         for nucleus in nuclei_sly4:
        
#             nucleus_name = nucleus[0]
#             Z = nucleus[1]
#             N = nucleus[2]
        
#             t0_nucleus = time.time()
        
#             # Single-nucleon
#             for nucleon in ('proton', 'neutron'):
            
#                 n_array = snmd.compute_momentum_distribution(
#                     q_array, nucleon, nucleus_name, Z, N, density, save=True)
        
#             # Pair
#             for pair in ('pn', 'pp', 'nn'):
        
#                 # Q > 0
#                 n_array = pmd.compute_momentum_distribution(
#                     q_array, Q_array, pair, nucleus_name, Z, N, density,
#                     save=True)
#                 # Q = 0
#                 n_Q0_array = pmd.compute_momentum_distribution_Q0(
#                         q_array, pair, nucleus_name, Z, N, density, save=True)

#             t1_nucleus = time.time()
#             mins = (t1_nucleus-t0_nucleus)/60
#             print(f'Done with {nucleus_name} after {mins:.2f} minutes.')
            
#         t1_lamb = time.time()
#         mins = (t1_lamb-t0_lamb)/60
#         if generator == 'Wegner':
#             print(f'Done with \lambda = {lamb} after {mins:.2f} minutes.\n')
#         else:
#             print(f'Done with \Lambda_BD = {lamb} after {mins:.2f}'
#                   ' minutes.\n')
        
#     t1_generator = time.time()
#     hours = (t1_generator-t0_generator)/3600
#     print(f'Done with generator = {generator} after {hours:.2f} hours.\n')

In [14]:
# generator = 'Wegner'
# lambdas_full = []
# for ilamb in np.arange(12, 1.0, -0.5):
#     if ilamb in (6.0, 3.0, 1.5):
#         pass
#     else:
#         lambdas_full.append(ilamb)
# example_nuclei = (('He4', 2, 2), ('Ca48', 20, 28), ('Pb208', 82, 126))

# for lamb in lambdas_full:
    
#     t0_lamb = time.time()
    
#     snmd = SingleNucleon(kvnn, kmax, kmid, ntot, channels, generator, lamb)
#     pmd = Pair(kvnn, kmax, kmid, ntot, channels, generator, lamb)
    
#     for nucleus in example_nuclei:
        
#         nucleus_name = nucleus[0]
#         Z = nucleus[1]
#         N = nucleus[2]
        
#         t0_nucleus = time.time()
        
#         # Single-nucleon
#         for nucleon in ('proton', 'neutron'):
            
#             n_array = snmd.compute_momentum_distribution(
#                 q_array, nucleon, nucleus_name, Z, N, density, save=True)
        
#         # Pair
#         for pair in ('pn', 'pp', 'nn'):
        
#             # Q > 0
#             n_array = pmd.compute_momentum_distribution(
#                 q_array, Q_array, pair, nucleus_name, Z, N, density,
#                 save=True)
#             # Q = 0
#             n_Q0_array = pmd.compute_momentum_distribution_Q0(
#                     q_array, pair, nucleus_name, Z, N, density, save=True)
            
#         t1_nucleus = time.time()
#         mins = (t1_nucleus-t0_nucleus)/60
#         print(f'Done with {nucleus_name} after {mins:.2f} minutes.')
        
#     t1_lamb = time.time()
#     mins = (t1_lamb-t0_lamb)/60
#     print(f'Done with \lambda = {lamb} after {mins:.2f} minutes.\n')